<a href="https://colab.research.google.com/github/vainaijr/few-shot-maml/blob/master/FewShot_maml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch-1.0.0a0+1d94a2b-cp36-cp36m-linux_x86_64.whl  \
  http://storage.googleapis.com/pytorch-tpu-releases/tf-1.13/torch_xla-0.1+5622d42-cp36-cp36m-linux_x86_64.whl

In [0]:
%%shell
pip install --upgrade tb-nightly

Requirement already up-to-date: tb-nightly in /usr/local/lib/python3.6/dist-packages (1.14.0a20190604)


In [0]:
import torch_xla
import torch_xla
import torch_xla_py.utils as xu
import torch_xla_py.xla_model as xm

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-06-05 09:22:44--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.9.93, 34.206.130.40, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.9.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  15.88M  34.5MB/s    in 0.5s    

2019-06-05 09:22:45 (34.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
LOG_DIR = '/gdrive/My\ Drive/runs/'
get_ipython().system_raw(
    "tensorboard --logdir {} --host 0.0.0.0 --port 6006 &".format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://56f635de.ngrok.io


In [0]:
from torch import cat, topk, bmm, stack, norm, zeros, ones, sum, transpose, save, load, manual_seed, cuda

import torch.nn as nn
from torch.nn import Module
from torch.nn import init
from torch.nn import Embedding
from torch.nn import Sequential
from torch.nn import Upsample
from torch.nn import ReflectionPad2d, ZeroPad2d
from torch.nn import Conv2d, ConvTranspose2d, Linear
from torch.nn import BatchNorm1d, BatchNorm2d, InstanceNorm2d
from torch.nn import PixelShuffle
from torch.nn import Dropout, Dropout2d
from torch.nn import LeakyReLU, ReLU, PReLU, Softmax, Tanh, Sigmoid
from torch.nn import AdaptiveAvgPool2d, AdaptiveMaxPool2d, MaxPool2d, AvgPool2d
from torch.nn import RNN, LSTM
from torch.nn import MSELoss, L1Loss, BCELoss, NLLLoss, BCEWithLogitsLoss, CrossEntropyLoss

from torch.nn.functional import relu6, avg_pool2d, softmax, interpolate, linear, conv2d, batch_norm
from torch.nn.functional import relu, sigmoid

from torch.autograd import grad, Variable as V
from torch.nn.utils import weight_norm 
from torch.optim import Adam, SGD, LBFGS
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from torch.utils.data import DataLoader, Dataset, sampler
from torch.utils.tensorboard import SummaryWriter
from torch.utils.checkpoint import checkpoint as cp

from torchvision import datasets
from torchvision.transforms import Compose, RandomHorizontalFlip, RandomResizedCrop, ToTensor, Normalize
from torchvision.transforms import CenterCrop, Resize, ColorJitter, ToPILImage, RandomCrop, RandomSizedCrop
from torchvision.models import ResNet, vgg19, vgg19_bn, densenet201, resnet101, resnet34, resnext101_32x8d
from torchvision.models import resnet152, resnet18, vgg16 
from torchvision.utils import make_grid, save_image


import math
import numpy as np
import torch.nn.functional as F
from torch.nn.utils.weight_norm import WeightNorm
from torchvision.utils import make_grid, save_image
from abc import abstractmethod
from PIL import ImageEnhance
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import torch.optim
import json
import torch.utils.data.sampler
import os
import glob
import random
import time
import h5py
import argparse
import torch.optim.lr_scheduler as lr_scheduler
from os import listdir
from os.path import isfile, isdir, join
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
print("CUDA available: ", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

identity = lambda x:x
step_ = 0
step__ = 0
stepp = 0
writer = SummaryWriter(log_dir='/gdrive/My Drive/runs/')
unloader = ToPILImage()
save_dir                    = '/gdrive/My Drive/save_dir_CUB/'
data_dir = {}
data_dir['CUB']             = '/gdrive/My Drive/CUB/' 
data_dir['miniImagenet']    = './filelists/miniImagenet/' 
data_dir['omniglot']        = './filelists/omniglot/' 
data_dir['emnist']          = './filelists/emnist/' 

CUDA available:  True


In [0]:
# This code is modified from https://github.com/facebookresearch/low-shot-shrink-hallucinate

# Basic ResNet model

def init_layer(L):
    # print("init_layer")
    classname = L.__class__.__name__
    # Initialization using fan-in
    if isinstance(L, nn.Conv2d):
        n = L.kernel_size[0]*L.kernel_size[1]*L.out_channels
        L.weight.data.normal_(0, math.sqrt(2.0/float(n)))
    elif isinstance(L, nn.BatchNorm2d):
        L.weight.data.fill_(1)
        L.bias.data.fill_(0)
    elif classname.find("Linear") != -1:
        init.normal_(L.weight.data, 0.0, 0.02)

class distLinear(nn.Module):
    def __init__(self, indim, outdim):
        super(distLinear, self).__init__()
        self.L = nn.Linear( indim, outdim, bias = False)
        self.class_wise_learnable_norm = True  #See the issue#4&8 in the github 
        if self.class_wise_learnable_norm:      
            WeightNorm.apply(self.L, 'weight', dim=0) #split the weight update component to direction and norm      

        if outdim <=200:
            self.scale_factor = 2; #a fixed scale factor to scale the output of cos value into a reasonably large input for softmax
        else:
            self.scale_factor = 10; #in omniglot, a larger scale factor is required to handle >1000 output classes.

    def forward(self, x):
        x_norm = torch.norm(x, p=2, dim =1).unsqueeze(1).expand_as(x)
        x_normalized = x.div(x_norm+ 0.00001)
        if not self.class_wise_learnable_norm:
            L_norm = torch.norm(self.L.weight.data, p=2, dim =1).unsqueeze(1).expand_as(self.L.weight.data)
            self.L.weight.data = self.L.weight.data.div(L_norm + 0.00001)
        cos_dist = self.L(x_normalized) #matrix product by forward function, but when using WeightNorm, this also multiply the cosine distance by a class-wise learnable norm, see the issue#4&8 in the github
        scores = self.scale_factor* (cos_dist) 

        return scores

class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):        
        return x.view(x.size(0), -1)


class Linear_fw(nn.Linear): #used in MAML to forward input with fast weight 
    def __init__(self, in_features, out_features):
        super(Linear_fw, self).__init__(in_features, out_features)
        self.weight.fast = None #Lazy hack to add fast weight link
        self.bias.fast = None
        print("__init__Linear_fw")

    def forward(self, x):
        if self.weight.fast is not None and self.bias.fast is not None:
            out = F.linear(x, self.weight.fast, self.bias.fast) #weight.fast (fast weight) is the temporaily adapted weight
            print("forward_Linear_fw_fast")
        else:
            print("forward_Linear_fw_1")
#             grid_ = make_grid(x)
            print("x.size() {}".format(x.size()))
            # writer.add_image('out_Linear_fw', x)
            out = super(Linear_fw, self).forward(x)
            print("out.size() {}".format(out.size()))
            print("forward_Linear_fw_2")
            
            
        return out

class Conv2d_fw(nn.Conv2d): #used in MAML to forward input with fast weight 
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,padding=0, bias = True):
        super(Conv2d_fw, self).__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=bias)
        self.weight.fast = None
        if not self.bias is None:
            self.bias.fast = None

    def forward(self, x):
        print("Conv2d_fw_x.size() {}".format(x.size()))
#         print("self.bias {}".format(self.bias))
#         print("self.weight.fast {}".format(self.weight.fast))
#         print("self.bias.fast {}".format(self.bias.fast))
        if self.bias is None:
            if self.weight.fast is not None:
                out = F.conv2d(x, self.weight.fast, None, stride= self.stride, padding=self.padding)
            else:
                out = super(Conv2d_fw, self).forward(x)
        else:
            if self.weight.fast is not None and self.bias.fast is not None:
                out = F.conv2d(x, self.weight.fast, self.bias.fast, stride= self.stride, padding=self.padding)
            else:
                out = super(Conv2d_fw, self).forward(x)
        
        T = out
        ER = T
        # print("T.unsqueeze {}".format(ER.unsqueeze(2).size()))
        grid___ = make_grid(T.view(out.size(0) * out.size(1), 1, out.size(2), out.size(3)))
        # print("out size {}".format(out.size()))
        # print("grid size {}".format(grid___.size()))
        
        writer.add_image("Conv2d_fw_1", grid___, 0, dataformats='CHW')
        
        
        # print("Conv2d_fw_out.size() {}".format(out.size()))
        return out
            
class BatchNorm2d_fw(nn.BatchNorm2d): #used in MAML to forward input with fast weight 
    def __init__(self, num_features):
        super(BatchNorm2d_fw, self).__init__(num_features)
        self.weight.fast = None
        self.bias.fast = None

    def forward(self, x):
        # print("BatchNorm2d_fw_x.size() {}".format(x.size()))
        running_mean = torch.zeros(x.data.size()[1]).cuda()
        running_var = torch.ones(x.data.size()[1]).cuda()
        if self.weight.fast is not None and self.bias.fast is not None:
            out = F.batch_norm(x, running_mean, running_var, self.weight.fast, self.bias.fast, training = True, momentum = 1)
            #batch_norm momentum hack: follow hack of Kate Rakelly in pytorch-maml/src/layers.py
        else:
            out = F.batch_norm(x, running_mean, running_var, self.weight, self.bias, training = True, momentum = 1)
        # print("BatchNorm2d_fw_out.size() {}".format(out.size()))
        return out

# Simple Conv Block
class ConvBlock(nn.Module):
    maml = False #Default
    def __init__(self, indim, outdim, pool = True, padding = 1):
        super(ConvBlock, self).__init__()
        self.indim  = indim
        self.outdim = outdim
        if self.maml:
            self.C      = Conv2d_fw(indim, outdim, 3, padding = padding)
            self.BN     = BatchNorm2d_fw(outdim)
        else:
            self.C      = nn.Conv2d(indim, outdim, 3, padding= padding)
            self.BN     = nn.BatchNorm2d(outdim)
        self.relu   = nn.ReLU(inplace=True)

        self.parametrized_layers = [self.C, self.BN, self.relu]
        if pool:
            self.pool   = nn.MaxPool2d(2)
            self.parametrized_layers.append(self.pool)

        for layer in self.parametrized_layers:
            init_layer(layer)

        self.trunk = nn.Sequential(*self.parametrized_layers)


    def forward(self,x):
        out = self.trunk(x)
#         grid = make_grid(out)
#         writer.add_image('ConvBlock', grid)
        return out

# # Simple ResNet Block
class SimpleBlock(nn.Module):
    maml = False #Default
    def __init__(self, indim, outdim, half_res):
        super(SimpleBlock, self).__init__()
        self.indim = indim
        self.outdim = outdim
        if self.maml:
            self.C1 = Conv2d_fw(indim, outdim, kernel_size=3, stride=2 if half_res else 1, padding=1, bias=False)
            self.BN1 = BatchNorm2d_fw(outdim)
            self.C2 = Conv2d_fw(outdim, outdim,kernel_size=3, padding=1,bias=False)
            self.BN2 = BatchNorm2d_fw(outdim)
        else:
            self.C1 = nn.Conv2d(indim, outdim, kernel_size=3, stride=2 if half_res else 1, padding=1, bias=False)
            self.BN1 = nn.BatchNorm2d(outdim)
            self.C2 = nn.Conv2d(outdim, outdim,kernel_size=3, padding=1,bias=False)
            self.BN2 = nn.BatchNorm2d(outdim)
        self.relu1 = nn.ReLU(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)

        self.parametrized_layers = [self.C1, self.C2, self.BN1, self.BN2]

        self.half_res = half_res

        # if the input number of channels is not equal to the output, then need a 1x1 convolution
        if indim!=outdim:
            if self.maml:
                self.shortcut = Conv2d_fw(indim, outdim, 1, 2 if half_res else 1, bias=False)
                self.BNshortcut = BatchNorm2d_fw(outdim)
            else:
                self.shortcut = nn.Conv2d(indim, outdim, 1, 2 if half_res else 1, bias=False)
                self.BNshortcut = nn.BatchNorm2d(outdim)

            self.parametrized_layers.append(self.shortcut)
            self.parametrized_layers.append(self.BNshortcut)
            self.shortcut_type = '1x1'
        else:
            self.shortcut_type = 'identity'

        for layer in self.parametrized_layers:
            init_layer(layer)

    def forward(self, x):
        out = self.C1(x)
        out = self.BN1(out)
        out = self.relu1(out)
        out = self.C2(out)
        out = self.BN2(out)
        short_out = x if self.shortcut_type == 'identity' else self.BNshortcut(self.shortcut(x))
        out = out + short_out
        out = self.relu2(out)
        return out



# # Bottleneck block
# class BottleneckBlock(nn.Module):
#     maml = False #Default
#     def __init__(self, indim, outdim, half_res):
#         super(BottleneckBlock, self).__init__()
#         bottleneckdim = int(outdim/4)
#         self.indim = indim
#         self.outdim = outdim
#         if self.maml:
#             self.C1 = Conv2d_fw(indim, bottleneckdim, kernel_size=1,  bias=False)
#             self.BN1 = BatchNorm2d_fw(bottleneckdim)
#             self.C2 = Conv2d_fw(bottleneckdim, bottleneckdim, kernel_size=3, stride=2 if half_res else 1,padding=1)
#             self.BN2 = BatchNorm2d_fw(bottleneckdim)
#             self.C3 = Conv2d_fw(bottleneckdim, outdim, kernel_size=1, bias=False)
#             self.BN3 = BatchNorm2d_fw(outdim)
#         else:
#             self.C1 = nn.Conv2d(indim, bottleneckdim, kernel_size=1,  bias=False)
#             self.BN1 = nn.BatchNorm2d(bottleneckdim)
#             self.C2 = nn.Conv2d(bottleneckdim, bottleneckdim, kernel_size=3, stride=2 if half_res else 1,padding=1)
#             self.BN2 = nn.BatchNorm2d(bottleneckdim)
#             self.C3 = nn.Conv2d(bottleneckdim, outdim, kernel_size=1, bias=False)
#             self.BN3 = nn.BatchNorm2d(outdim)

#         self.relu = nn.ReLU()
#         self.parametrized_layers = [self.C1, self.BN1, self.C2, self.BN2, self.C3, self.BN3]
#         self.half_res = half_res


#         # if the input number of channels is not equal to the output, then need a 1x1 convolution
#         if indim!=outdim:
#             if self.maml:
#                 self.shortcut = Conv2d_fw(indim, outdim, 1, stride=2 if half_res else 1, bias=False)
#             else:
#                 self.shortcut = nn.Conv2d(indim, outdim, 1, stride=2 if half_res else 1, bias=False)

#             self.parametrized_layers.append(self.shortcut)
#             self.shortcut_type = '1x1'
#         else:
#             self.shortcut_type = 'identity'

#         for layer in self.parametrized_layers:
#             init_layer(layer)


#     def forward(self, x):

#         short_out = x if self.shortcut_type == 'identity' else self.shortcut(x)
#         out = self.C1(x)
#         out = self.BN1(out)
#         out = self.relu(out)
#         out = self.C2(out)
#         out = self.BN2(out)
#         out = self.relu(out)
#         out = self.C3(out)
#         out = self.BN3(out)
#         out = out + short_out

#         out = self.relu(out)
#         return out


class ConvNet(nn.Module):
    def __init__(self, depth, flatten = True):
        super(ConvNet,self).__init__()
        trunk = []
        for i in range(depth):
            indim = 3 if i == 0 else 64
            outdim = 64
            B = ConvBlock(indim, outdim, pool = ( i <4 ) ) #only pooling for fist 4 layers
            trunk.append(B)

        if flatten:
            trunk.append(Flatten())
        # print("ConvNet_init")
        self.trunk = nn.Sequential(*trunk)
        self.final_feat_dim = 1600
      
    def forward(self,x):
        # print("ConvNet_forward")
        # print('x size {}'.format(x.size()))
        
        
        grid = make_grid(x)
        writer.add_image('x_', grid)
        out = self.trunk(x)
        # print('out size {}'.format(out.size()))
        
#         plt.figure(figsize=(8,8))
#         plt.axis("off")
#         plt.imshow(out.data.cpu().numpy())
#         plt.show()

#         grid = make_grid(out)
#         writer.add_image('ConvNet', grid)
        return out

# class ConvNetNopool(nn.Module): #Relation net use a 4 layer conv with pooling in only first two layers, else no pooling
#     def __init__(self, depth):
#         super(ConvNetNopool,self).__init__()
#         trunk = []
#         for i in range(depth):
#             indim = 3 if i == 0 else 64
#             outdim = 64
#             B = ConvBlock(indim, outdim, pool = ( i in [0,1] ), padding = 0 if i in[0,1] else 1  ) #only first two layer has pooling and no padding
#             trunk.append(B)

#         self.trunk = nn.Sequential(*trunk)
#         self.final_feat_dim = [64,19,19]

#     def forward(self,x):
#         out = self.trunk(x)
#         return out

# class ConvNetS(nn.Module): #For omniglot, only 1 input channel, output dim is 64
#     def __init__(self, depth, flatten = True):
#         super(ConvNetS,self).__init__()
#         trunk = []
#         for i in range(depth):
#             indim = 1 if i == 0 else 64
#             outdim = 64
#             B = ConvBlock(indim, outdim, pool = ( i <4 ) ) #only pooling for fist 4 layers
#             trunk.append(B)

#         if flatten:
#             trunk.append(Flatten())

#         self.trunk = nn.Sequential(*trunk)
#         self.final_feat_dim = 64

#     def forward(self,x):
#         out = x[:,0:1,:,:] #only use the first dimension
#         out = self.trunk(out)
#         return out

# class ConvNetSNopool(nn.Module): #Relation net use a 4 layer conv with pooling in only first two layers, else no pooling. For omniglot, only 1 input channel, output dim is [64,5,5]
#     def __init__(self, depth):
#         super(ConvNetSNopool,self).__init__()
#         trunk = []
#         for i in range(depth):
#             indim = 1 if i == 0 else 64
#             outdim = 64
#             B = ConvBlock(indim, outdim, pool = ( i in [0,1] ), padding = 0 if i in[0,1] else 1  ) #only first two layer has pooling and no padding
#             trunk.append(B)

#         self.trunk = nn.Sequential(*trunk)
#         self.final_feat_dim = [64,5,5]

#     def forward(self,x):
#         out = x[:,0:1,:,:] #only use the first dimension
#         out = self.trunk(out)
#         return out

# class ResNet(nn.Module):
#     maml = False #Default
#     def __init__(self,block,list_of_num_layers, list_of_out_dims, flatten = True):
#         # list_of_num_layers specifies number of layers in each stage
#         # list_of_out_dims specifies number of output channel for each stage
#         super(ResNet,self).__init__()
#         assert len(list_of_num_layers)==4, 'Can have only four stages'
#         if self.maml:
#             conv1 = Conv2d_fw(3, 64, kernel_size=7, stride=2, padding=3,
#                                                bias=False)
#             bn1 = BatchNorm2d_fw(64)
#         else:
#             conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
#                                                bias=False)
#             bn1 = nn.BatchNorm2d(64)

#         relu = nn.ReLU()
#         pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

#         init_layer(conv1)
#         init_layer(bn1)


#         trunk = [conv1, bn1, relu, pool1]

#         indim = 64
#         for i in range(4):

#             for j in range(list_of_num_layers[i]):
#                 half_res = (i>=1) and (j==0)
#                 B = block(indim, list_of_out_dims[i], half_res)
#                 trunk.append(B)
#                 indim = list_of_out_dims[i]

#         if flatten:
#             avgpool = nn.AvgPool2d(7)
#             trunk.append(avgpool)
#             trunk.append(Flatten())
#             self.final_feat_dim = indim
#         else:
#             self.final_feat_dim = [ indim, 7, 7]

#         self.trunk = nn.Sequential(*trunk)

#     def forward(self,x):
#         out = self.trunk(x)
#         return out

def Conv4():
    print("ConvNet(4)")
    return ConvNet(4)

# def Conv6():
#     print("ConvNet(6)")
#     return ConvNet(6)

# def Conv4NP():
#     return ConvNetNopool(4)

# def Conv6NP():
#     return ConvNetNopool(6)

  
# def Conv4S():
#     return ConvNetS(4)

# def Conv4SNP():
#     return ConvNetSNopool(4)

# def ResNet10( flatten = True):
#     return ResNet(SimpleBlock, [1,1,1,1],[64,128,256,512], flatten)

# def ResNet18( flatten = True):
#     return ResNet(SimpleBlock, [2,2,2,2],[64,128,256,512], flatten)

# def ResNet34( flatten = True):
#     return ResNet(SimpleBlock, [3,4,6,3],[64,128,256,512], flatten)

# def ResNet50( flatten = True):
#     return ResNet(BottleneckBlock, [3,4,6,3], [256,512,1024,2048], flatten)

# def ResNet101( flatten = True):
#     return ResNet(BottleneckBlock, [3,4,23,3],[256,512,1024,2048], flatten)





In [0]:
class MetaTemplate(nn.Module):
    def __init__(self, model_func, n_way, n_support, change_way = True):
        super().__init__()
        # print("__init__")
        self.n_way      = n_way
        self.n_support  = n_support
        self.n_query    = -1 #(change depends on input) 
        self.feature    = model_func()
        self.feat_dim   = self.feature.final_feat_dim
        self.change_way = change_way  #some methods allow different_way classification during training and test
        

    @abstractmethod
    def set_forward(self,x,is_feature):
        pass

    @abstractmethod
    def set_forward_loss(self, x):
        pass

    def forward(self,x):
        # print("forward")
        out  = self.feature.forward(x)
        return out

    def parse_feature(self,x,is_feature):
        x    = V(x.cuda())
        
        print("parse_feature_x.size() {}".format(x.size()))
        
        if is_feature:
            z_all = x
        else:
            x           = x.contiguous().view( self.n_way * (self.n_support + self.n_query), *x.size()[2:]) 
            z_all       = self.feature.forward(x)
            z_all       = z_all.view( self.n_way, self.n_support + self.n_query, -1)
        z_support   = z_all[:, :self.n_support]
        z_query     = z_all[:, self.n_support:]
        
#         global step__
#         step__ += 1
#         x_ = make_grid(z_support)
#         writer.add_image('support_data', x_, step__)
        
#         y_ = make_grid(z_query)
#         writer.add_image('query_data', y_, step__) 

        return z_support, z_query

    def correct(self, x):       
        scores = self.set_forward(x)
        y_query = np.repeat(range( self.n_way ), self.n_query )
        print("correct")
        topk_scores, topk_labels = scores.data.topk(1, 1, True, True)
        topk_ind = topk_labels.cpu().numpy()
        top1_correct = np.sum(topk_ind[:,0] == y_query)
        return float(top1_correct), len(y_query)

    def train_loop(self, epoch, train_loader, optimizer ):
        print_freq = 10
        global stepp
        avg_loss=0
        print("train_loop")
        for i, (x,_ ) in enumerate(train_loader):
            self.n_query = x.size(1) - self.n_support           
            if self.change_way:
                self.n_way  = x.size(0)
            optimizer.zero_grad()
            loss = self.set_forward_loss( x )
            loss.backward()
            optimizer.step()
            avg_loss = avg_loss+loss.item()

            if i % print_freq==0:
                #print(optimizer.state_dict()['param_groups'][0]['lr'])
                print('Epoch {:d} | Batch {:d}/{:d} | Loss {:f}'.format(epoch, i, len(train_loader), 
                                                                        avg_loss/float(i+1)))
                stepp += 1
                writer.add_scalar('loss_epoch', avg_loss/float(i+1), stepp)
                

    def test_loop(self, test_loader, record = None):
        correct =0
        count = 0
        acc_all = []
        print("test_loop")
        iter_num = len(test_loader) 
        for i, (x,_) in enumerate(test_loader):
            self.n_query = x.size(1) - self.n_support
            if self.change_way:
                self.n_way  = x.size(0)
            correct_this, count_this = self.correct(x)
            acc_all.append(correct_this/ count_this*100  )

        acc_all  = np.asarray(acc_all)
        acc_mean = np.mean(acc_all)
        acc_std  = np.std(acc_all)
        
        print('%d Test Acc = %4.2f%% +- %4.2f%%' %(iter_num,  acc_mean, 1.96* acc_std/np.sqrt(iter_num)))

        return acc_mean

    def set_forward_adaptation(self, x, is_feature = True): #further adaptation, default is fixing feature and train a new softmax clasifier
        assert is_feature == True, 'Feature is fixed in further adaptation'
        print("set_forward_adaptation")
        z_support, z_query  = self.parse_feature(x,is_feature)

        z_support   = z_support.contiguous().view(self.n_way* self.n_support, -1 )
        z_query     = z_query.contiguous().view(self.n_way* self.n_query, -1 )

        y_support = torch.from_numpy(np.repeat(range( self.n_way ), self.n_support ))
        y_support = V(y_support.cuda())

        linear_clf = Linear(self.feat_dim, self.n_way)
        linear_clf = linear_clf.cuda()

        set_optimizer = SGD(linear_clf.parameters(), lr = 0.01, momentum=0.9, dampening=0.9, weight_decay=0.001)

        loss_function = CrossEntropyLoss()
        loss_function = loss_function.cuda()
        
        batch_size = 4
        support_size = self.n_way* self.n_support
        for epoch in range(100):
            rand_id = np.random.permutation(support_size)
            for i in range(0, support_size , batch_size):
                set_optimizer.zero_grad()
                selected_id = torch.from_numpy( rand_id[i: min(i+batch_size, support_size) ]).cuda()
                z_batch = z_support[selected_id]
                y_batch = y_support[selected_id] 
                scores = linear_clf(z_batch)
                loss = loss_function(scores,y_batch)
                loss.backward()

        scores = linear_clf(z_query)
        return scores

In [0]:

# Copyright 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.


transformtypedict=dict(Brightness=ImageEnhance.Brightness, Contrast=ImageEnhance.Contrast, Sharpness=ImageEnhance.Sharpness, Color=ImageEnhance.Color)



class ImageJitter(object):
    def __init__(self, transformdict):
        self.transforms = [(transformtypedict[k], transformdict[k]) for k in transformdict]


    def __call__(self, img):
        out = img
        randtensor = torch.rand(len(self.transforms))

        for i, (transformer, alpha) in enumerate(self.transforms):
            r = alpha*(randtensor[i]*2.0 -1.0) + 1
            out = transformer(out).enhance(r).convert('RGB')

        return out





In [0]:


# This code is modified from https://github.com/facebookresearch/low-shot-shrink-hallucinate

class TransformLoader:
    def __init__(self, image_size, 
                 normalize_param    = dict(mean= [0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225]),
                 jitter_param       = dict(Brightness=0.4, Contrast=0.4, Color=0.4)):
        self.image_size = image_size
        self.normalize_param = normalize_param
        self.jitter_param = jitter_param
    
    def parse_transform(self, transform_type):
        if transform_type=='ImageJitter':
            method = add_transforms.ImageJitter( self.jitter_param )
            return method
        method = getattr(transforms, transform_type)
        if transform_type=='RandomSizedCrop':
            return method(self.image_size) 
        elif transform_type=='CenterCrop':
            return method(self.image_size) 
        elif transform_type=='Resize':
            return method([int(self.image_size*1.15), int(self.image_size*1.15)])
        elif transform_type=='Normalize':
            return method(**self.normalize_param )
        else:
            return method()

    def get_composed_transform(self, aug = False):
        if aug:
            transform_list = ['RandomSizedCrop', 'ImageJitter', 'RandomHorizontalFlip', 'ToTensor', 'Normalize']
        else:
            transform_list = ['Resize','CenterCrop', 'ToTensor', 'Normalize']

        transform_funcs = [ self.parse_transform(x) for x in transform_list]
        transform = transforms.Compose(transform_funcs)
        return transform

class DataManager:
    @abstractmethod
    def get_data_loader(self, data_file, aug):
        pass 


class SimpleDataManager(DataManager):
    def __init__(self, image_size, batch_size):        
        super(SimpleDataManager, self).__init__()
        self.batch_size = batch_size
        self.trans_loader = TransformLoader(image_size)

    def get_data_loader(self, data_file, aug): #parameters that would change on train/val set
        transform = self.trans_loader.get_composed_transform(aug)
        dataset = SimpleDataset(data_file, transform)
        data_loader_params = dict(batch_size = self.batch_size, shuffle = True, num_workers = 12, pin_memory = True)       
        data_loader = torch.utils.data.DataLoader(dataset, **data_loader_params)

        return data_loader

class SetDataManager(DataManager):
    def __init__(self, image_size, n_way, n_support, n_query, n_eposide =100):        
        super(SetDataManager, self).__init__()
        self.image_size = image_size
        self.n_way = n_way
        self.batch_size = n_support + n_query
        self.n_eposide = n_eposide

        self.trans_loader = TransformLoader(image_size)

    def get_data_loader(self, data_file, aug): #parameters that would change on train/val set
        transform = self.trans_loader.get_composed_transform(aug)
        dataset = SetDataset( data_file , self.batch_size, transform )
        sampler = EpisodicBatchSampler(len(dataset), self.n_way, self.n_eposide )  
        data_loader_params = dict(batch_sampler = sampler,  num_workers = 12, pin_memory = True)       
        data_loader = torch.utils.data.DataLoader(dataset, **data_loader_params)
        return data_loader


In [0]:
class SimpleDataset:
    def __init__(self, data_file, transform, target_transform=identity):
        with open(data_file, 'r') as f:
            self.meta = json.load(f)
        self.transform = transform
        self.target_transform = target_transform


    def __getitem__(self,i):
        image_path = os.path.join(self.meta['image_names'][i])
        img = Image.open(image_path).convert('RGB')
        img = self.transform(img)
        target = self.target_transform(self.meta['image_labels'][i])
        return img, target

    def __len__(self):
        return len(self.meta['image_names'])


class SetDataset:
    def __init__(self, data_file, batch_size, transform):
        with open(data_file, 'r') as f:
            self.meta = json.load(f)
 
        self.cl_list = np.unique(self.meta['image_labels']).tolist()

        self.sub_meta = {}
        for cl in self.cl_list:
            self.sub_meta[cl] = []

        for x,y in zip(self.meta['image_names'],self.meta['image_labels']):
            self.sub_meta[y].append(x)

        self.sub_dataloader = [] 
        sub_data_loader_params = dict(batch_size = batch_size,
                                  shuffle = True,
                                  num_workers = 0, #use main thread only or may receive multiple batches
                                  pin_memory = False)        
        for cl in self.cl_list:
            sub_dataset = SubDataset(self.sub_meta[cl], cl, transform = transform )
            self.sub_dataloader.append( torch.utils.data.DataLoader(sub_dataset, **sub_data_loader_params) )

    def __getitem__(self,i):
        return next(iter(self.sub_dataloader[i]))

    def __len__(self):
        return len(self.cl_list)

class SubDataset:
    def __init__(self, sub_meta, cl, transform=transforms.ToTensor(), target_transform=identity):
        self.sub_meta = sub_meta
        self.cl = cl 
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self,i):
        #print( '%d -%d' %(self.cl,i))
        image_path = os.path.join( self.sub_meta[i])
        img = Image.open(image_path).convert('RGB')
        img = self.transform(img)
        target = self.target_transform(self.cl)
        return img, target

    def __len__(self):
        return len(self.sub_meta)

class EpisodicBatchSampler(object):
    def __init__(self, n_classes, n_way, n_episodes):
        self.n_classes = n_classes
        self.n_way = n_way
        self.n_episodes = n_episodes

    def __len__(self):
        return self.n_episodes

    def __iter__(self):
        for i in range(self.n_episodes):
            yield torch.randperm(self.n_classes)[:self.n_way]

In [0]:
class SimpleHDF5Dataset:
    def __init__(self, file_handle = None):
        if file_handle == None:
            self.f = ''
            self.all_feats_dset = []
            self.all_labels = []
            self.total = 0 
        else:
            self.f = file_handle
            self.all_feats_dset = self.f['all_feats'][...]
            self.all_labels = self.f['all_labels'][...]
            self.total = self.f['count'][0]
           # print('here')
    def __getitem__(self, i):
        return torch.Tensor(self.all_feats_dset[i,:]), int(self.all_labels[i])

    def __len__(self):
        return self.total

def init_loader(filename):
    with h5py.File(filename, 'r') as f:
        fileset = SimpleHDF5Dataset(f)

    #labels = [ l for l  in fileset.all_labels if l != 0]
    feats = fileset.all_feats_dset
    labels = fileset.all_labels
    while np.sum(feats[-1]) == 0:
        feats  = np.delete(feats,-1,axis = 0)
        labels = np.delete(labels,-1,axis = 0)
        
    class_list = np.unique(np.array(labels)).tolist() 
    inds = range(len(labels))

    cl_data_file = {}
    for cl in class_list:
        cl_data_file[cl] = []
    for ind in inds:
        cl_data_file[labels[ind]].append( feats[ind])

    return cl_data_file

In [0]:

def one_hot(y, num_class):         
    return torch.zeros((len(y), num_class)).scatter_(1, y.unsqueeze(1), 1)

def DBindex(cl_data_file):
    class_list = cl_data_file.keys()
    cl_num= len(class_list)
    cl_means = []
    stds = []
    DBs = []
    for cl in class_list:
        cl_means.append( np.mean(cl_data_file[cl], axis = 0) )
        stds.append( np.sqrt(np.mean( np.sum(np.square( cl_data_file[cl] - cl_means[-1]), axis = 1))))

    mu_i = np.tile( np.expand_dims( np.array(cl_means), axis = 0), (len(class_list),1,1) )
    mu_j = np.transpose(mu_i,(1,0,2))
    mdists = np.sqrt(np.sum(np.square(mu_i - mu_j), axis = 2))
    
    for i in range(cl_num):
        DBs.append( np.max([ (stds[i]+ stds[j])/mdists[i,j]  for j in range(cl_num) if j != i ]) )
    return np.mean(DBs)

def sparsity(cl_data_file):
    class_list = cl_data_file.keys()
    cl_sparsity = []
    for cl in class_list:
        cl_sparsity.append(np.mean([np.sum(x!=0) for x in cl_data_file[cl] ])  ) 

    return np.mean(cl_sparsity) 

In [0]:

model_dict = dict(
    Conv4 = Conv4,
#             Conv4S = Conv4S,
#             Conv6 = Conv6,
#             ResNet10 = ResNet10,
#             ResNet18 = ResNet18,
#             ResNet34 = ResNet34,
#             ResNet50 = ResNet50,
#             ResNet101 = ResNet101
    )

def get_assigned_file(checkpoint_dir,num):
    assign_file = os.path.join(checkpoint_dir, '{:d}.tar'.format(num))
    return assign_file

def get_resume_file(checkpoint_dir):
    filelist = glob.glob(os.path.join(checkpoint_dir, '*.tar'))
    if len(filelist) == 0:
        return None

    filelist =  [ x  for x in filelist if os.path.basename(x) != 'best_model.tar' ]
    epochs = np.array([int(os.path.splitext(os.path.basename(x))[0]) for x in filelist])
    max_epoch = np.max(epochs)
    resume_file = os.path.join(checkpoint_dir, '{:d}.tar'.format(max_epoch))
    return resume_file

def get_best_file(checkpoint_dir):    
    best_file = os.path.join(checkpoint_dir, 'best_model.tar')
    if os.path.isfile(best_file):
        return best_file
    else:
        return get_resume_file(checkpoint_dir)

In [0]:
# This code is modified from https://github.com/dragen1860/MAML-Pytorch and https://github.com/katerakelly/pytorch-maml 

class MAML(MetaTemplate):
    def __init__(self, model_func,  n_way, n_support, approx = False):
        super(MAML, self).__init__( model_func,  n_way, n_support, change_way = False)

        self.loss_fn = CrossEntropyLoss()
        self.classifier = Linear_fw(self.feat_dim, n_way)
        self.classifier.bias.data.fill_(0)
        # self.conv2d_fw = Conv2d_fw(self.)
        self.n_task     = 4
        self.task_update_num = 5
        self.train_lr = 0.01
        self.approx = approx #first order approx.        
        # print("__init__MAML")

    def forward(self,x):
        print("forward_MAML_1")
        # print("MAML_forward_x.size() {}".format(x.size()))
        out  = self.feature.forward(x)
        # print("MAML_forward_out.size() {}".format(out.size()))
        # writer.add_image('feature', out)
        # print("forward_MAML_2")
        scores  = self.classifier.forward(out)
        # print("MAML_forward_scores.size() {}".format(scores.size()))
        writer.add_embedding(scores, metadata=None, label_img=None, global_step=0, tag='default',
                             metadata_header=None)
        # writer.add_image('classifier', scores)
        # print("forward_MAML_3")
        return scores

    def set_forward(self,x, is_feature = False):
        assert is_feature == False, 'MAML do not support fixed feature' 
        print("set_forward_MAML")
        x = x.cuda()
        x_var = V(x)
        
        x_a_i = x_var[:,:self.n_support,:,:,:].contiguous().view( self.n_way* self.n_support, *x.size()[2:]) #support data 
        x_b_i = x_var[:,self.n_support:,:,:,:].contiguous().view( self.n_way* self.n_query,   *x.size()[2:]) #query data
        y_a_i = V( torch.from_numpy( np.repeat(range( self.n_way ), self.n_support ) )).cuda() #label for support data
        # print("x_var[:,:self.n_support,:,:,:].size() {}".format(x_var[:,:self.n_support,:,:,:].size()))
        # print("x_var[:,self.n_support:,:,:,:].size() {}".format(x_var[:,self.n_support:,:,:,:].size()))
        # print("x_a_i.size() {}".format(x_a_i.size()))
        # print("x_b_i.size() {}".format(x_b_i.size()))
        global step__
        step__ = 1
        x_ = make_grid(x_a_i)
        writer.add_image('support_data', x_, step__)
        
        y_ = make_grid(x_b_i)
        writer.add_image('query_data', y_, step__)
        

        
        fast_parameters = list(self.parameters()) #the first gradient calcuated in line 45 is based on original weight
        for weight in self.parameters():
            weight.fast = None
        self.zero_grad()

        for task_step in range(self.task_update_num):
            # print("set_forward_MAML_1")
            scores = self.forward(x_a_i)
            # print("set_forward_MAML_2")
            set_loss = self.loss_fn( scores, y_a_i)
            # print("set_forward_MAML_3")
            grad = torch.autograd.grad(set_loss, fast_parameters, create_graph=True) #build full graph support gradient of gradient
            # print("set_forward_MAML_4")
            if self.approx:
                grad = [ g.detach()  for g in grad ] #do not calculate gradient of gradient if using first order approximation
            # print("set_forward_MAML_5")
            fast_parameters = []
            for k, weight in enumerate(self.parameters()):
                #for usage of weight.fast, please see Linear_fw, Conv_fw in backbone.py 
                # print("set_forward_MAML_6")
                if weight.fast is None:
                    weight.fast = weight - self.train_lr * grad[k] #create weight.fast 
                else:
                    weight.fast = weight.fast - self.train_lr * grad[k] #create an updated weight.fast, note the '-' is not merely minus value, but to create a new weight.fast 
                fast_parameters.append(weight.fast) #gradients calculated in line 45 are based on newest fast weight, but the graph will retain the link to old weight.fasts
        global step_
        step_ += 1
        writer.add_histogram('weight.fast', weight.fast, step_)
        # print("set_forward_MAML______")
        scores = self.forward(x_b_i)
        return scores

    def set_forward_adaptation(self,x, is_feature = False): #overwrite parrent function
        print("set_forward_adaptation_MAML")
        raise ValueError('MAML performs further adapation simply by increasing task_upate_num')


    def set_forward_loss(self, x):
        print("set_forward_loss_MAML")
        scores = self.set_forward(x, is_feature = False)
        y_b_i = V( torch.from_numpy( np.repeat(range( self.n_way ), self.n_query   ) )).cuda()
        loss = self.loss_fn(scores, y_b_i)

        return loss

    def train_loop(self, epoch, train_loader, optimizer): #overwrite parrent function
        print_freq = 10
        avg_loss=0
        task_count = 0
        loss_all = []
        optimizer.zero_grad()

        print("train_loop_MAML")
        #train
        for i, (x,_) in enumerate(train_loader):
            self.n_query = x.size(1) - self.n_support
            print("x.size() {}".format(x.size()))
            print("n_query {}".format(self.n_query))
            print("x.size(1) {}".format(x.size(1)))
            print("n_support {}".format(self.n_support))
            assert self.n_way  ==  x.size(0), "MAML do not support way change"

            loss = self.set_forward_loss(x)
            avg_loss = avg_loss+loss.item()
            loss_all.append(loss)

            task_count += 1
            print("task_count {}".format(task_count))

            if task_count == self.n_task: #MAML update several tasks at one time
                loss_q = torch.stack(loss_all).sum(0)
                loss_q.backward()

                optimizer.step()
                task_count = 0
                loss_all = []
            optimizer.zero_grad()
            if i % print_freq==0:
                print('Epoch {:d} | Batch {:d}/{:d} | Loss {:f}'.format(epoch, i, len(train_loader), avg_loss/float(i+1)))
                writer.add_scalar('loss_epoch_', avg_loss/float(i+1), epoch)
                      
    def test_loop(self, test_loader, return_std = False): #overwrite parrent function
        correct =0
        count = 0
        acc_all = []
        print("test_loop_MAML")
        iter_num = len(test_loader) 
        for i, (x,_) in enumerate(test_loader):
            self.n_query = x.size(1) - self.n_support
            assert self.n_way  ==  x.size(0), "MAML do not support way change"
            correct_this, count_this = self.correct(x)
            acc_all.append(correct_this/ count_this *100 )

        acc_all  = np.asarray(acc_all)
        acc_mean = np.mean(acc_all)
        acc_std  = np.std(acc_all)
        print('%d Test Acc = %4.2f%% +- %4.2f%%' %(iter_num,  acc_mean, 1.96* acc_std/np.sqrt(iter_num)))
        if return_std:
            return acc_mean, acc_std
        else:
            return acc_mean


In [0]:
class RelationNet(MetaTemplate):
    def __init__(self, model_func,  n_way, n_support, loss_type = 'mse'):
        super(RelationNet, self).__init__(model_func,  n_way, n_support)
        
        print("RelationNet__init__")
        self.loss_type = loss_type  #'softmax'# 'mse'
        self.relation_module = RelationModule( self.feat_dim , 8, self.loss_type ) #relation net features are not pooled, so self.feat_dim is [dim, w, h] 

        if self.loss_type == 'mse':
            self.loss_fn = MSELoss()  
        else:
            self.loss_fn = CrossEntropyLoss()

    def set_forward(self,x,is_feature = False):
        print("RelationNet__set_forward_x.size() {}".format(x.size()))
        z_support, z_query  = self.parse_feature(x,is_feature)
        print("RelationNet__set_forward__")
        z_support   = z_support.contiguous()
        z_proto     = z_support.view( self.n_way, self.n_support, *self.feat_dim ).mean(1) 
        z_query     = z_query.contiguous().view( self.n_way* self.n_query, *self.feat_dim )

        
        z_proto_ext = z_proto.unsqueeze(0).repeat(self.n_query* self.n_way,1,1,1,1)
        z_query_ext = z_query.unsqueeze(0).repeat( self.n_way,1,1,1,1)
        z_query_ext = torch.transpose(z_query_ext,0,1)
        extend_final_feat_dim = self.feat_dim.copy()
        extend_final_feat_dim[0] *= 2
        relation_pairs = torch.cat((z_proto_ext,z_query_ext),2).view(-1, *extend_final_feat_dim)
        relations = self.relation_module(relation_pairs).view(-1, self.n_way)

        return relations

    def set_forward_adaptation(self,x,is_feature = True): #overwrite parent function
        assert is_feature == True, 'Finetune only support fixed feature' 
        print("RelationNet__set_forward_adaptation__")
        full_n_support = self.n_support
        full_n_query = self.n_query
        relation_module_clone = RelationModule( self.feat_dim , 8, self.loss_type )
        relation_module_clone.load_state_dict(self.relation_module.state_dict())
        z_support, z_query  = self.parse_feature(x,is_feature)
        z_support   = z_support.contiguous()
        set_optimizer = SGD(self.relation_module.parameters(), lr = 0.01, momentum=0.9, dampening=0.9, weight_decay=0.001)

        self.n_support = 3
        self.n_query = 2

        z_support_cpu = z_support.data.cpu().numpy()
        for epoch in range(100):
            perm_id = np.random.permutation(full_n_support).tolist()            
            sub_x = np.array([z_support_cpu[i,perm_id,:,:,:] for i in range(z_support.size(0))])
            sub_x = torch.Tensor(sub_x).cuda()
            if self.change_way:
                self.n_way  = sub_x.size(0)
            set_optimizer.zero_grad()
            y = torch.from_numpy(np.repeat(range( self.n_way ), self.n_query ))
            scores = self.set_forward(sub_x, is_feature = True)
            if self.loss_type == 'mse':
                y_oh = one_hot(y, self.n_way)
                y_oh = V(y_oh.cuda())            

                loss =  self.loss_fn(scores, y_oh )
            else:
                y = V(y.cuda())
                loss = self.loss_fn(scores, y )
            loss.backward()
            set_optimizer.step()

        self.n_support = full_n_support
        self.n_query = full_n_query
        z_proto     = z_support.view( self.n_way, self.n_support, *self.feat_dim ).mean(1) 
        z_query     = z_query.contiguous().view( self.n_way* self.n_query, *self.feat_dim )

        
        z_proto_ext = z_proto.unsqueeze(0).repeat(self.n_query* self.n_way,1,1,1,1)
        z_query_ext = z_query.unsqueeze(0).repeat( self.n_way,1,1,1,1)
        z_query_ext = torch.transpose(z_query_ext,0,1)
        extend_final_feat_dim = self.feat_dim.copy()
        extend_final_feat_dim[0] *= 2
        relation_pairs = torch.cat((z_proto_ext,z_query_ext),2).view(-1, *extend_final_feat_dim)
        relations = self.relation_module(relation_pairs).view(-1, self.n_way)

        self.relation_module.load_state_dict(relation_module_clone.state_dict())
        return relations
    def set_forward_loss(self, x):
      
        y = torch.from_numpy(np.repeat(range( self.n_way ), self.n_query ))
        print("RelationNet__set_forward_loss__")
        scores = self.set_forward(x)
        if self.loss_type == 'mse':
            y_oh = one_hot(y, self.n_way)
            y_oh = V(y_oh.cuda())            

            return self.loss_fn(scores, y_oh )
        else:
            y = V(y.cuda())
            return self.loss_fn(scores, y )

class RelationConvBlock(Module):
    def __init__(self, indim, outdim, padding = 0):
        super(RelationConvBlock, self).__init__()
        self.indim  = indim
        self.outdim = outdim
        self.C      = Conv2d(indim, outdim, 3, padding = padding )
        self.BN     = BatchNorm2d(outdim, momentum=1, affine=True)
        self.relu   = ReLU()
        self.pool   = MaxPool2d(2)
        print("RelationConvBlock__init__")
        self.parametrized_layers = [self.C, self.BN, self.relu, self.pool]

        for layer in self.parametrized_layers:
            init_layer(layer)

        self.trunk = Sequential(*self.parametrized_layers)

    def forward(self,x):
        print("RelationConvBlock__forward__x.size() {}".format(x.size()))
        out = self.trunk(x)
        print("RelationConvBlock__forward__out.size() {}".format(out.size()))
        return out

class RelationModule(Module):
    """docstring for RelationNetwork"""
    def __init__(self,input_size,hidden_size, loss_type = 'mse'):        
        super(RelationModule, self).__init__()
        print("RelationModule__init__")
        self.loss_type = loss_type
        padding = 1 if ( input_size[1] <10 ) and ( input_size[2] <10 ) else 0 # when using Resnet, conv map without avgpooling is 7x7, need padding in block to do pooling

        self.layer1 = RelationConvBlock(input_size[0]*2, input_size[0], padding = padding )
        self.layer2 = RelationConvBlock(input_size[0], input_size[0], padding = padding )

        shrink_s = lambda s: int((int((s- 2 + 2*padding)/2)-2 + 2*padding)/2)

        self.fc1 = Linear( input_size[0]* shrink_s(input_size[1]) * shrink_s(input_size[2]), hidden_size )
        self.fc2 = Linear( hidden_size,1)

    def forward(self,x):
        print("RelationModule__forward__x.size() {}".format(x.size()))
        out = self.layer1(x)
        print("RelationModule__forward__out_layer1.size() {}".format(out.size()))
        out = self.layer2(out)
        print("RelationModule__forward__out_layer2.size() {}".format(out.size()))
        out = out.view(out.size(0),-1)
        print("RelationModule__forward__out_layer2_view.size() {}".format(out.size()))
        out = F.relu(self.fc1(out))
        print("RelationModule__forward__out_fc1_relu.size() {}".format(out.size()))
        if self.loss_type == 'mse':
            out = F.sigmoid(self.fc2(out))
            print("RelationModule__forward__out_fc2_sigmoid.size() {}".format(out.size()))
        elif self.loss_type == 'softmax':
            out = self.fc2(out)
            print("RelationModule__forward__out_fc2.size() {}".format(out.size()))

        return out

In [0]:
def train(base_loader, val_loader, model, optimization, start_epoch, stop_epoch, params):    
    if optimization == 'Adam':
        optimizer = Adam(model.parameters())
    else:
       raise ValueError('Unknown optimization, please define by yourself')
    writer.add_graph(model, (torch.rand(25, 3, 84, 84).to(device),))
    # writer.add_graph()
    max_acc = 0       
    print("train!!!")
    for epoch in range(start_epoch,stop_epoch):
        model.train()
        print("train_")
        model.train_loop(epoch, base_loader,  optimizer ) #model are called by reference, no need to return 
        print("train__")
        model.eval()
        print("train___")
        if not os.path.isdir(params.checkpoint_dir):
            os.makedirs(params.checkpoint_dir)

        acc = model.test_loop( val_loader)
        if acc > max_acc : #for baseline and baseline++, we don't use validation here so we let acc = -1
            print("best model! save...")
            max_acc = acc
            outfile = os.path.join(params.checkpoint_dir, 'best_model.tar')
            torch.save({'epoch':epoch, 'state':model.state_dict()}, outfile)

        if (epoch % params.save_freq==0) or (epoch==stop_epoch-1):
            outfile = os.path.join(params.checkpoint_dir, '{:d}.tar'.format(epoch))
            torch.save({'epoch':epoch, 'state':model.state_dict()}, outfile)

    return model


np.random.seed(10)
script = 'train'

parser = argparse.ArgumentParser(description= 'few-shot script %s' %(script))
parser.add_argument('--dataset'     , default='CUB',        help='CUB/miniImagenet/cross/omniglot/cross_char')
parser.add_argument('--model'       , default='Conv4',      help='model: Conv{4|6} / ResNet{10|18|34|50|101}') # 50 and 101 are not used in the paper
parser.add_argument('--method'      , default='maml_approx',   help='baseline/baseline++/protonet/matchingnet/relationnet{_softmax}/maml{_approx}') #relationnet_softmax replace L2 norm with softmax to expedite training, maml_approx use first-order approximation in the gradient for efficiency
parser.add_argument('--train_n_way' , default=5, type=int,  help='class num to classify for training') #baseline and baseline++ would ignore this parameter
parser.add_argument('--test_n_way'  , default=5, type=int,  help='class num to classify for testing (validation) ') #baseline and baseline++ only use this parameter in finetuning
parser.add_argument('--n_shot'      , default=5, type=int,  help='number of labeled data in each class, same as n_support') #baseline and baseline++ only use this parameter in finetuning
parser.add_argument('--train_aug'   , action='store_true',  help='perform data augmentation or not during training ') #still required for save_features.py and test.py to find the model path correctly

if script == 'train':
    parser.add_argument('--num_classes' , default=200, type=int, help='total number of classes in softmax, only used in baseline') #make it larger than the maximum label value in base class
    parser.add_argument('--save_freq'   , default=50, type=int, help='Save frequency')
    parser.add_argument('--start_epoch' , default=0, type=int,help ='Starting epoch')
    parser.add_argument('--stop_epoch'  , default=-1, type=int, help ='Stopping epoch') #for meta-learning methods, each epoch contains 100 episodes. The default epoch number is dataset dependent. See train.py
    parser.add_argument('--resume'      , default=True, action='store_true', help='continue from previous trained model with largest epoch')
    parser.add_argument('--warmup'      , action='store_true', help='continue from baseline, neglected if resume is true') #never used in the paper
elif script == 'save_features':
    parser.add_argument('--split'       , default='novel', help='base/val/novel') #default novel, but you can also test base/val class accuracy if you want 
    parser.add_argument('--save_iter', default=-1, type=int,help ='save feature from the model trained in x epoch, use the best model if x is -1')
elif script == 'test':
    parser.add_argument('--split'       , default='novel', help='base/val/novel') #default novel, but you can also test base/val class accuracy if you want 
    parser.add_argument('--save_iter', default=-1, type=int,help ='saved feature from the model trained in x epoch, use the best model if x is -1')
    parser.add_argument('--adaptation'  , action='store_true', help='further adaptation in test time or not')
else:
   raise ValueError('Unknown script')

params = parser.parse_args('')


if params.dataset == 'cross':
    base_file = data_dir['miniImagenet'] + 'all.json' 
    val_file   = data_dir['CUB'] + 'val.json' 
elif params.dataset == 'cross_char':
    base_file = data_dir['omniglot'] + 'noLatin.json' 
    val_file   = data_dir['emnist'] + 'val.json' 
else:
    base_file = data_dir[params.dataset] + 'base.json' 
    val_file   = data_dir[params.dataset] + 'val.json' 

if 'Conv' in params.model:
    if params.dataset in ['omniglot', 'cross_char']:
        image_size = 28
    else:
        image_size = 84
else:
    image_size = 224

if params.dataset in ['omniglot', 'cross_char']:
    assert params.model == 'Conv4' and not params.train_aug ,'omniglot only support Conv4 without augmentation'
    params.model = 'Conv4S'

optimization = 'Adam'

if params.stop_epoch == -1: 
    if params.method in ['baseline', 'baseline++'] :
        if params.dataset in ['omniglot', 'cross_char']:
            params.stop_epoch = 5
        elif params.dataset in ['CUB']:
            params.stop_epoch = 200 # This is different as stated in the open-review paper. However, using 400 epoch in baseline actually lead to over-fitting
        elif params.dataset in ['miniImagenet', 'cross']:
            params.stop_epoch = 400
        else:
            params.stop_epoch = 400 #default
    else: #meta-learning methods
        if params.n_shot == 1:
            params.stop_epoch = 600
        elif params.n_shot == 5:
            params.stop_epoch = 400
        else:
            params.stop_epoch = 600 #default


if params.method in ['baseline', 'baseline++'] :
    base_datamgr    = SimpleDataManager(image_size, batch_size = 16)
    base_loader     = base_datamgr.get_data_loader( base_file , aug = params.train_aug )
    val_datamgr     = SimpleDataManager(image_size, batch_size = 64)
    val_loader      = val_datamgr.get_data_loader( val_file, aug = False)

    if params.dataset == 'omniglot':
        assert params.num_classes >= 4112, 'class number need to be larger than max label id in base class'
    if params.dataset == 'cross_char':
        assert params.num_classes >= 1597, 'class number need to be larger than max label id in base class'

    if params.method == 'baseline':
        model           = BaselineTrain( model_dict[params.model], params.num_classes)
    elif params.method == 'baseline++':
        model           = BaselineTrain( model_dict[params.model], params.num_classes, loss_type = 'dist')

elif params.method in ['protonet','matchingnet','relationnet', 'relationnet_softmax', 'maml', 'maml_approx']:
    n_query = max(1, int(16* params.test_n_way/params.train_n_way)) #if test_n_way is smaller than train_n_way, reduce n_query to keep batch size small

    train_few_shot_params    = dict(n_way = params.train_n_way, n_support = params.n_shot) 
    base_datamgr            = SetDataManager(image_size, n_query = n_query,  **train_few_shot_params)
    base_loader             = base_datamgr.get_data_loader( base_file , aug = params.train_aug )

    test_few_shot_params     = dict(n_way = params.test_n_way, n_support = params.n_shot) 
    val_datamgr             = SetDataManager(image_size, n_query = n_query, **test_few_shot_params)
    val_loader              = val_datamgr.get_data_loader( val_file, aug = False) 
    #a batch for SetDataManager: a [n_way, n_support + n_query, dim, w, h] tensor        
    print("hello")
    if params.method == 'protonet':
        model           = ProtoNet( model_dict[params.model], **train_few_shot_params )
    elif params.method == 'matchingnet':
        model           = MatchingNet( model_dict[params.model], **train_few_shot_params )
    elif params.method in ['relationnet', 'relationnet_softmax']:
        if params.model == 'Conv4': 
            feature_model = Conv4NP
        elif params.model == 'Conv6': 
            feature_model = Conv6NP
        elif params.model == 'Conv4S': 
            feature_model = Conv4SNP
        else:
            feature_model = lambda: model_dict[params.model]( flatten = False )
        loss_type = 'mse' if params.method == 'relationnet' else 'softmax'

        model           = RelationNet( feature_model, loss_type = loss_type , **train_few_shot_params )
    elif params.method in ['maml' , 'maml_approx']:
        ConvBlock.maml = True
        # SimpleBlock.maml = True
        # BottleneckBlock.maml = True
        # ResNet.maml = True
        model           = MAML(  model_dict[params.model], approx = (params.method == 'maml_approx') , **train_few_shot_params )
        if params.dataset in ['omniglot', 'cross_char']: #maml use different parameter in omniglot
            model.n_task     = 32
            model.task_update_num = 1
            model.train_lr = 0.1
else:
   raise ValueError('Unknown method')

print("hello!")
model = model.cuda()

params.checkpoint_dir = '%s/checkpoints/%s/%s_%s' %(save_dir, params.dataset, params.model, params.method)
if params.train_aug:
    params.checkpoint_dir += '_aug'
if not params.method  in ['baseline', 'baseline++']: 
    params.checkpoint_dir += '_%dway_%dshot' %( params.train_n_way, params.n_shot)

if not os.path.isdir(params.checkpoint_dir):
    os.makedirs(params.checkpoint_dir)

start_epoch = params.start_epoch
stop_epoch = params.stop_epoch
if params.method == 'maml' or params.method == 'maml_approx' :
    stop_epoch = params.stop_epoch * model.n_task #maml use multiple tasks in one update 

if params.resume:
    resume_file = get_resume_file(params.checkpoint_dir)
    if resume_file is not None:
        tmp = torch.load(resume_file)
        start_epoch = tmp['epoch']+1
        model.load_state_dict(tmp['state'])
elif params.warmup: #We also support warmup from pretrained baseline feature, but we never used in our paper
    baseline_checkpoint_dir = '%s/checkpoints/%s/%s_%s' %(save_dir, params.dataset, params.model, 'baseline')
    if params.train_aug:
        baseline_checkpoint_dir += '_aug'
    warmup_resume_file = get_resume_file(baseline_checkpoint_dir)
    tmp = torch.load(warmup_resume_file)
    if tmp is not None: 
        state = tmp['state']
        state_keys = list(state.keys())
        for i, key in enumerate(state_keys):
            if "feature." in key:
                newkey = key.replace("feature.","")  # an architecture model has attribute 'feature', load architecture feature to backbone by casting name from 'feature.trunk.xx' to 'trunk.xx'  
                state[newkey] = state.pop(key)
            else:
                state.pop(key)
        model.feature.load_state_dict(state)
    else:
        raise ValueError('No warm_up file')

model = train(base_loader, val_loader,  model, optimization, start_epoch, stop_epoch, params)


hello
ConvNet(4)
__init__Linear_fw
hello!
forward_MAML_1
Conv2d_fw_x.size() torch.Size([25, 3, 84, 84])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:78: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


Conv2d_fw_x.size() torch.Size([25, 64, 42, 42])
Conv2d_fw_x.size() torch.Size([25, 64, 21, 21])
Conv2d_fw_x.size() torch.Size([25, 64, 10, 10])
forward_Linear_fw_1
x.size() torch.Size([25, 1600])
out.size() torch.Size([25, 5])
forward_Linear_fw_2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


train!!!
train_
train_loop_MAML
x.size() torch.Size([5, 21, 3, 84, 84])
n_query 16
x.size(1) 21
n_support 5
set_forward_loss_MAML
set_forward_MAML
forward_MAML_1
Conv2d_fw_x.size() torch.Size([25, 3, 84, 84])
Conv2d_fw_x.size() torch.Size([25, 64, 42, 42])
Conv2d_fw_x.size() torch.Size([25, 64, 21, 21])
Conv2d_fw_x.size() torch.Size([25, 64, 10, 10])
forward_Linear_fw_1
x.size() torch.Size([25, 1600])
out.size() torch.Size([25, 5])
forward_Linear_fw_2
forward_MAML_1
Conv2d_fw_x.size() torch.Size([25, 3, 84, 84])
Conv2d_fw_x.size() torch.Size([25, 64, 42, 42])
Conv2d_fw_x.size() torch.Size([25, 64, 21, 21])
Conv2d_fw_x.size() torch.Size([25, 64, 10, 10])
forward_Linear_fw_fast
forward_MAML_1
Conv2d_fw_x.size() torch.Size([25, 3, 84, 84])
Conv2d_fw_x.size() torch.Size([25, 64, 42, 42])
Conv2d_fw_x.size() torch.Size([25, 64, 21, 21])
Conv2d_fw_x.size() torch.Size([25, 64, 10, 10])
forward_Linear_fw_fast
forward_MAML_1
Conv2d_fw_x.size() torch.Size([25, 3, 84, 84])
Conv2d_fw_x.size() torch

KeyboardInterrupt: ignored

In [0]:
log_fn = xm.get_log_fn()
torch.set_default_tensor_type('torch.FloatTensor')
for epoch in range(1, num_epochs + 1):
  xla_model.train(train_loader, optimizer, batch_size, log_interval=log_interval, metrics_debug=False,
                  log_fn=log_fn)
  accuracy = xla_model.test(test_loader, xm.category_eval_fn(F.nll_loss), batch_size, log_fn=log_fn)

In [0]:
def feature_evaluation(cl_data_file, model, n_way = 5, n_support = 5, n_query = 15, adaptation = False):
    class_list = cl_data_file.keys()

    select_class = random.sample(class_list,n_way)
    z_all  = []
    for cl in select_class:
        img_feat = cl_data_file[cl]
        perm_ids = np.random.permutation(len(img_feat)).tolist()
        z_all.append( [ np.squeeze( img_feat[perm_ids[i]]) for i in range(n_support+n_query) ] )     # stack each batch

    z_all = torch.from_numpy(np.array(z_all) )
    model.n_query = n_query
    if adaptation:
        scores  = model.set_forward_adaptation(z_all, is_feature = True)
    else:
        scores  = model.set_forward(z_all, is_feature = True)
    pred = scores.data.cpu().numpy().argmax(axis = 1)
    y = np.repeat(range( n_way ), n_query )
    acc = np.mean(pred == y)*100 
    return acc

if __name__ == '__main__':
    params = parse_args('test')

    acc_all = []

    iter_num = 600

    few_shot_params = dict(n_way = params.test_n_way , n_support = params.n_shot) 

    if params.dataset in ['omniglot', 'cross_char']:
        assert params.model == 'Conv4' and not params.train_aug ,'omniglot only support Conv4 without augmentation'
        params.model = 'Conv4S'

    if params.method == 'baseline':
        model           = BaselineFinetune( model_dict[params.model], **few_shot_params )
    elif params.method == 'baseline++':
        model           = BaselineFinetune( model_dict[params.model], loss_type = 'dist', **few_shot_params )
    elif params.method == 'protonet':
        model           = ProtoNet( model_dict[params.model], **few_shot_params )
    elif params.method == 'matchingnet':
        model           = MatchingNet( model_dict[params.model], **few_shot_params )
    elif params.method in ['relationnet', 'relationnet_softmax']:
        if params.model == 'Conv4': 
            feature_model = Conv4NP
        elif params.model == 'Conv6': 
            feature_model = Conv6NP
        elif params.model == 'Conv4S': 
            feature_model = Conv4SNP
        else:
            feature_model = lambda: model_dict[params.model]( flatten = False )
        loss_type = 'mse' if params.method == 'relationnet' else 'softmax'
        model           = RelationNet( feature_model, loss_type = loss_type , **few_shot_params )
    elif params.method in ['maml' , 'maml_approx']:
        ConvBlock.maml = True
        SimpleBlock.maml = True
        BottleneckBlock.maml = True
        ResNet.maml = True
        model = MAML(  model_dict[params.model], approx = (params.method == 'maml_approx') , **few_shot_params )
        if params.dataset in ['omniglot', 'cross_char']: #maml use different parameter in omniglot
            model.n_task     = 32
            model.task_update_num = 1
            model.train_lr = 0.1
    else:
       raise ValueError('Unknown method')

    model = model.cuda()

    checkpoint_dir = '%s/checkpoints/%s/%s_%s' %(save_dir, params.dataset, params.model, params.method)
    if params.train_aug:
        checkpoint_dir += '_aug'
    if not params.method in ['baseline', 'baseline++'] :
        checkpoint_dir += '_%dway_%dshot' %( params.train_n_way, params.n_shot)

    #modelfile   = get_resume_file(checkpoint_dir)

    if not params.method in ['baseline', 'baseline++'] : 
        if params.save_iter != -1:
            modelfile   = get_assigned_file(checkpoint_dir,params.save_iter)
        else:
            modelfile   = get_best_file(checkpoint_dir)
        if modelfile is not None:
            tmp = torch.load(modelfile)
            model.load_state_dict(tmp['state'])

    split = params.split
    if params.save_iter != -1:
        split_str = split + "_" +str(params.save_iter)
    else:
        split_str = split
    if params.method in ['maml', 'maml_approx']: #maml do not support testing with feature
        if 'Conv' in params.model:
            if params.dataset in ['omniglot', 'cross_char']:
                image_size = 28
            else:
                image_size = 84 
        else:
            image_size = 224

        datamgr         = SetDataManager(image_size, n_eposide = iter_num, n_query = 15 , **few_shot_params)
        
        if params.dataset == 'cross':
            if split == 'base':
                loadfile = data_dir['miniImagenet'] + 'all.json' 
            else:
                loadfile   = data_dir['CUB'] + split +'.json'
        elif params.dataset == 'cross_char':
            if split == 'base':
                loadfile = data_dir['omniglot'] + 'noLatin.json' 
            else:
                loadfile  = data_dir['emnist'] + split +'.json' 
        else: 
            loadfile    = data_dir[params.dataset] + split + '.json'

        novel_loader     = datamgr.get_data_loader( loadfile, aug = False)
        if params.adaptation:
            model.task_update_num = 100 #We perform adaptation on MAML simply by updating more times.
        model.eval()
        acc_mean, acc_std = model.test_loop( novel_loader, return_std = True)

    else:
        novel_file = os.path.join( checkpoint_dir.replace("checkpoints","features"), split_str +".hdf5") #defaut split = novel, but you can also test base or val classes
        cl_data_file = feat_loader.init_loader(novel_file)

        for i in range(iter_num):
            acc = feature_evaluation(cl_data_file, model, n_query = 15, adaptation = params.adaptation, **few_shot_params)
            acc_all.append(acc)

        acc_all  = np.asarray(acc_all)
        acc_mean = np.mean(acc_all)
        acc_std  = np.std(acc_all)
        print('%d Test Acc = %4.2f%% +- %4.2f%%' %(iter_num, acc_mean, 1.96* acc_std/np.sqrt(iter_num)))
    with open('./record/results.txt' , 'a') as f:
        timestamp = time.strftime("%Y%m%d-%H%M%S", time.localtime()) 
        aug_str = '-aug' if params.train_aug else ''
        aug_str += '-adapted' if params.adaptation else ''
        if params.method in ['baseline', 'baseline++'] :
            exp_setting = '%s-%s-%s-%s%s %sshot %sway_test' %(params.dataset, split_str, params.model, params.method, aug_str, params.n_shot, params.test_n_way )
        else:
            exp_setting = '%s-%s-%s-%s%s %sshot %sway_train %sway_test' %(params.dataset, split_str, params.model, params.method, aug_str , params.n_shot , params.train_n_way, params.test_n_way )
        acc_str = '%d Test Acc = %4.2f%% +- %4.2f%%' %(iter_num, acc_mean, 1.96* acc_std/np.sqrt(iter_num))
        f.write( 'Time: %s, Setting: %s, Acc: %s \n' %(timestamp,exp_setting,acc_str)  )

In [0]:
!ls

In [0]:
!sh ./filelists/CUB/download_CUB.sh

In [0]:
!mv ../gdrive/My\ Drive/CUB_200_2011 ../gdrive/My\ Drive/CUB/

In [0]:

cwd = os.getcwd() 
data_path = './drive/My Drive/CUB/images/'
savedir = './drive/My Drive/CUB/'
dataset_list = ['base','val','novel']

#if not os.path.exists(savedir):
#    os.makedirs(savedir)

folder_list = [f for f in listdir(data_path) if isdir(join(data_path, f))]
folder_list.sort()
label_dict = dict(zip(folder_list,range(0,len(folder_list))))

classfile_list_all = []

for i, folder in enumerate(folder_list):
    folder_path = join(data_path, folder)
    classfile_list_all.append( [ join(folder_path, cf) for cf in listdir(folder_path) if (isfile(join(folder_path,cf)) and cf[0] != '.')])
    random.shuffle(classfile_list_all[i])


for dataset in dataset_list:
    file_list = []
    label_list = []
    for i, classfile_list in enumerate(classfile_list_all):
        if 'base' in dataset:
            if (i%2 == 0):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()
        if 'val' in dataset:
            if (i%4 == 1):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()
        if 'novel' in dataset:
            if (i%4 == 3):
                file_list = file_list + classfile_list
                label_list = label_list + np.repeat(i, len(classfile_list)).tolist()

    fo = open(savedir + dataset + ".json", "w")
    fo.write('{"label_names": [')
    fo.writelines(['"%s",' % item  for item in folder_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write('],')

    fo.write('"image_names": [')
    fo.writelines(['"%s",' % item  for item in file_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write('],')

    fo.write('"image_labels": [')
    fo.writelines(['%d,' % item  for item in label_list])
    fo.seek(0, os.SEEK_END) 
    fo.seek(fo.tell()-1, os.SEEK_SET)
    fo.write(']}')

    fo.close()
    print("%s -OK" %dataset)


base -OK
val -OK
novel -OK


In [0]:
!mv novel.json ./filelists/CUB/ 

In [0]:
!cp -r ../gdrive/My\ Drive/CUB_200_2011/ ./CUB_200_2011/

^C


In [0]:
writer.add_image?

In [0]:
make_grid?

Object `IPython.core` not found.
